<a href="https://colab.research.google.com/github/CarolineAndradeR/Python/blob/master/Teste_Hip%C3%B3teses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INTRODUÇÃO - TEÓRIA**

**Hipótese** - Transformar questionamentos em afirmações passíveis de testes.

Metodologia que permite verificar se os dados da amostra trazem evidências para rejeitar ou não rejeitar a hipótese proposta.

Uma hipótese será testada somente se esxistirem dados disponíveis, quando não há dados se torna uma hipótese pouco plausível no momento, uma vez que não pode ser testada. Porém pode recorrer aos processos de amostragem para obter esses dados e realizar os testes possíveis.

**Níveis** 
*   **Descritivo** - é a probalilidade de obter um efeito igual ou maior do que o esperado pela hipótese, esse efeito pode ser a diferença entre dois grupos, ex: o site com fundo escuro possui maior taxa de conversão do que o site com o fundo branco.
*   **Valor Crítico** - é um determinado ponto na distribução de probalidade do teste que precisa ser comparado com o valor observado no teste, caso esse valor seja muito diferente, há evidência para rejeitar a hipótese testada.

**PROCEDIMENTOS**
1.   Fizar a hipótese nula e a hipótese alternativa.
2.   Definir qual estimador será utilizado (média, variância).
3.   Fixar o nível de confiança (90%, 95%, 99%) para definir o valor crítico.
4.   Utilizar os dados da amostra obtida para calcular o valor da estatística teste.
5.   Rejeitar a hipótese nula se o valor da estatística de teste for maior que o valor definido na etapa 3.

**Hipótese nula e alternativa:**
*   **Nula** - Deve sempre colocar um sinal de igual ou combinado (igual, maior igual, menor igual...)
*   **Alternativa** - Um complemento da hipótese nula, ex: se o sinal da hipótese nula for igual na hipótese alternativa se usa o valor difere. (sinal ao contrario da hipótese nula).


# **Prático**

In [ ]:
# Instalação de uma versão mais atualizada do Scipy
! sudo pip install scipy==1.7.1

In [ ]:
# Instalação nova ferramenta para teste de hipoteses
! sudo pip install pingouin

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from scipy import stats as st
import pingouin as pg

In [ ]:
dados_ab = pd.read_csv('exampleDataABtest.csv')
dados_ab.head()

In [ ]:
dados_nps = pd.read_csv('/content/nps_example.csv', sep = ';')
dados_nps.head()

In [ ]:
#Teste T
#Comparar os grupos

dados_ab.groupby('group') \
        .agg(media_cliques = pd.NamedAgg('clickedTrue', 'mean'),
             dp_cliques = pd.NamedAgg('clickedTrue', 'std'),
             n = pd.NamedAgg('clickedTrue', 'count'))


In [8]:
grA = dados_ab[dados_ab['group'] == 'A']['clickedTrue']
grB = dados_ab[dados_ab['group'] == 'B']['clickedTrue']

In [11]:
#Teste T - com scipy 
#alternative - tipo de hipotese 
#two-sided - paramentro de diferente entre os grupos (grupo a é diferente do grupo b)
st.ttest_ind(a=grA, b=grB, alternative='two-sided')

#Se o p-value < 0,05 - então rejeito a hipotese nula (a é diferente do grupo b)

Ttest_indResult(statistic=-2.669938469060931, pvalue=0.007709783987515948)

In [13]:
#Teste T - Pingouin
pg.ttest(x=grA,
         y=grB,
         alternative='two-sided',
         confidence=0.95)
#confidence - nível de confiança

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.669938,998,two-sided,0.00771,"[-0.07, -0.01]",0.168862,2.349,0.760344


In [15]:
# Teste F - Quando temos mais de dois grupo para ser comparados
dados_nps.groupby('response_status') \
          .size() \
          .to_frame('n') \
          .reset_index()

,response_status,n
0,Complete,2281
1,Incomplete,265
2,Terminated,33


In [ ]:
#Verificando se a coluna em especifico existe algum valor nulo
dados_nps[dados_nps['nps_score'].isnull()]

In [ ]:
#Filtado a coluna buscando todas com status completo e não está com nps nulo
dados_nps_filtrados = dados_nps[(dados_nps['response_status'] == 'Complete') & \
                                (dados_nps['nps_score'].notna())]
dados_nps_filtrados.head()                                

In [ ]:
dados_nps_filtrados.groupby('age') \
                    .agg(media_nps = pd.NamedAgg('nps_score', 'mean'),
                         dp_nps = pd.NamedAgg('nps_score', 'std'),
                         n = pd.NamedAgg('nps_score', 'size')) \
                    .reset_index()

In [21]:
# Quando em um grupo temos um número muito pequeno de indivíduos devemos desconsiderar esse grupo
# Filtrando o grupo com poucas pessoas
# != - Rejeitas dados

dados_nps_filtrados_aj = dados_nps_filtrados[dados_nps_filtrados['age'] != '75+']

# Dependendo da regra de négocio esse grupo rejeitado poderia ser unido com outro grupo, ex: 65+ 

In [23]:
#Teste F - Scipy
#Separar os grupos
dados_18_24 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '18-24']['nps_score']
dados_25_34 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '25-34']['nps_score']
dados_35_44 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '35-44']['nps_score']
dados_45_54 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '45-54']['nps_score']
dados_55_64 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '55-64']['nps_score']
dados_65_74 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '65-74']['nps_score']

#Realizar teste
st.f_oneway(dados_18_24,
            dados_25_34,
            dados_35_44,
            dados_45_54,
            dados_55_64,
            dados_65_74)

F_onewayResult(statistic=3.5221660981040768, pvalue=0.0035606861304276695)

In [27]:
#Teste F - Pingouin
#Primeira diferença entre Scipy e que a linha de codigos é menor
#anova - Analise de Variance
pg.anova(dv ='nps_score',
         between = 'age',
         data = dados_nps_filtrados_aj,
         detailed = True)

#p-unc = pvalue
#Se o p-unc < 0,05 - então rejeito a hipotese nula, alguma das médias é diferente das demais

,Source,SS,DF,MS,F,p-unc,np2
0,age,16.888794,5,3.377759,3.522166,0.003561,0.00777
1,Within,2156.791916,2249,0.959000,NaN,NaN,NaN
